# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [48]:
# Write your code below.
%load_ext dotenv
%dotenv 

from dotenv import load_dotenv
load_dotenv()

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [40]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [73]:
import os
from glob import glob

# Write your code below.
dask.config.set({'dataframe.query-planning': True})
price_data_dir = os.getenv('PRICE_DATA')
print('PRICE_DATA',price_data_dir)

# Use glob to find all parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_dir,"**/*.parquet/*.parquet"))

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

print(dd_px.columns)


PRICE_DATA ../../05_src/data/prices/
Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [77]:
# Write your code below.
import numpy as np
from dask.dataframe import rolling

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), 
                       Adj_close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


/var/folders/vb/q1bpp1t50n338507pp0hqf440000gn/T/ipykernel_19670/3073771561.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [85]:
# Write your code below.
pd_feat = dd_feat.compute()
pd_feat.reset_index(inplace=True)
pd_feat['rolling_avg_return'] = pd_feat.groupby('ticker')['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

print(pd_feat.head())

  ticker       Date        Open        High         Low       Close  \
0    WAT 2018-01-02  194.119995  196.639999  193.600006  195.690002   
1    WAT 2018-01-03  195.350006  197.940002  195.350006  197.770004   
2    WAT 2018-01-04  198.470001  200.029999  198.360001  199.660004   
3    WAT 2018-01-05  200.679993  202.850006  200.000000  202.229996   
4    WAT 2018-01-08  202.279999  205.289993  200.679993  204.759995   

    Adj Close  Volume       sector                       subsector  year  \
0  195.690002  355300  Health Care  Life Sciences Tools & Services  2018   
1  197.770004  271700  Health Care  Life Sciences Tools & Services  2018   
2  199.660004  337400  Health Care  Life Sciences Tools & Services  2018   
3  202.229996  363300  Health Care  Life Sciences Tools & Services  2018   
4  204.759995  483100  Health Care  Life Sciences Tools & Services  2018   

    Close_lag  Adj_close_lag   returns  hi_lo_range  rolling_avg_return  
0         NaN            NaN       NaN    

It is no neccessary to convert to pandas to calculate. 
It is better to do it in Dask, the benefit of doing in Dask is, the code is more simple and consistent, it avoid to switch between different data libraries and ensure all transformation are in good order. Moreover, Dask operation like rolling() and .mean() designed to handle large datasets.

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.